In [1]:
import pandas as pd
import numpy as np
import altair as alt
covid = pd.read_csv("C:/Users/kosti/OneDrive/Desktop/School/Econ Classes/EC 499/Data/Strigency/Strigency.csv")
gdp = pd.read_csv("C:/Users/kosti/OneDrive/Desktop/School/Econ Classes/EC 499/Data/GDP/AllStateGDP.csv")
energy = pd.read_excel("C:/Users/kosti/OneDrive/Desktop/UMICH/SI/SI 671/Project/HS861M 2010-.xlsx")

In [2]:
# Removing rows where 'Year' or 'Month' columns contain non-numeric data
data_monthly_states_cleaned = energy[pd.to_numeric(energy['Year'], errors='coerce').notnull()]
data_monthly_states_cleaned = data_monthly_states_cleaned[pd.to_numeric(data_monthly_states_cleaned['Month'], errors='coerce').notnull()]

# Convert 'Year' and 'Month' columns to integer to prepare for combining them into a date
data_monthly_states_cleaned['Year'] = data_monthly_states_cleaned['Year'].astype(int)
data_monthly_states_cleaned['Month'] = data_monthly_states_cleaned['Month'].astype(int)

# Combining 'Year' and 'Month' into a single 'Date' column
data_monthly_states_cleaned['Date'] = pd.to_datetime(data_monthly_states_cleaned[['Year', 'Month']].assign(DAY=1))

energy = data_monthly_states_cleaned
# Create a long form DataFrame for the energy data with 'state' 'Date', 'Sector', and 'Value' columns

energy_long = energy.melt(id_vars=['State', 'Date'], value_vars=['Industrial Sales', 'Transportation Sales', 'Residential Sales', 'Commercial Sales'], var_name='Sector', value_name='Value')


nevada = energy_long[energy_long['State'] == 'NV']
utah = energy_long[energy_long['State'] == 'UT']

In [3]:
# Delete MaxRowsError
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
unemployment = pd.read_csv("Unemployment in America Per US State.csv")
unemployment.head()
unemployment['Date'] = pd.to_datetime(unemployment['Year'].astype(str) + '-' + unemployment['Month'].astype(str))
features = ['State/Area', 'Date', 'Total Civilian Non-Institutional Population in State/Area']
population = unemployment[features]

#rename columns
population = population.rename(columns={'State/Area': 'State', 'Total Civilian Non-Institutional Population in State/Area': 'Population'})
# state to two letter abbreviation using dictionary

In [5]:

state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

population['State'] = population['State'].map(state_abbrev)
# Merge the data on state and date
df2 = pd.merge(energy, population, how='inner', on=['State', 'Date'])

df2['Population'] = df2['Population'].str.replace(',', '').astype(float)

# Sales for each sector and GDP per capita

df2['Residential Sales per Capita'] = df2['Residential Sales'] / df2['Population']
df2['Commercial Sales per Capita'] = df2['Commercial Sales'] / df2['Population']
df2['Industrial Sales per Capita'] = df2['Industrial Sales'] / df2['Population']
df2['Transportation Sales per Capita'] = df2['Transportation Sales'] / df2['Population']


In [6]:
# Create a long form DataFrame for the energy data with 'state' 'Date', 'Sector', and 'Value' per capita columns

df2_long = df2.melt(id_vars=['State', 'Date'], value_vars=['Residential Sales per Capita', 'Commercial Sales per Capita', 'Industrial Sales per Capita', 'Transportation Sales per Capita'], var_name='Sector', value_name='Value')
# filter years 2019-current
df2_long = df2_long[df2_long['Date'].dt.year >= 2019]

In [7]:
nevada_avg = nevada.groupby('Sector')['Value'].sum().reset_index()
nevada_avg['Value'] = nevada_avg['Value'] / nevada_avg['Value'].sum() * 100

nevada_avg


utah_avg = utah.groupby('Sector')['Value'].sum().reset_index()
utah_avg['Value'] = utah_avg['Value'] / utah_avg['Value'].sum() * 100

utah_avg

,Sector,Value
0,Commercial Sales,37.774467
1,Industrial Sales,30.186357
2,Residential Sales,31.879307
3,Transportation Sales,0.159868


In [8]:
import us

def get_state_name_from_abbr(abbr):
    state = us.states.lookup(abbr)
    return state.name if state else None

df2_long['State'] = df2_long['State'].apply(get_state_name_from_abbr)


# Calculate the total value for each state and date
df2_long['Total'] = df2_long.groupby(['State', 'Date'])['Value'].transform('sum')

# Calculate the percentage
df2_long['Percentage'] = (df2_long['Value'] / df2_long['Total']) * 100

In [9]:
# Calculate the overall average percentage for each sector
average_percentage = (
    df2_long.groupby(['Sector', 'State'])['Percentage']
    .mean()
    .reset_index()
    .rename(columns={'Percentage': 'AveragePercentage'})
)

average_percentage


df2_long = pd.merge(df2_long, average_percentage, on=['Sector', 'State'])
df2_long

,State,Date,Sector,Value,Total,Percentage,AveragePercentage
0,Alaska,2022-12-01,Residential Sales per Capita,0.411168,1.058780,38.834168,34.160059
1,Alaska,2022-11-01,Residential Sales per Capita,0.340059,0.953188,35.675999,34.160059
2,Alaska,2022-10-01,Residential Sales per Capita,0.290495,0.885506,32.805486,34.160059
3,Alaska,2022-09-01,Residential Sales per Capita,0.260635,0.844391,30.866677,34.160059
4,Alaska,2022-08-01,Residential Sales per Capita,0.265998,0.863188,30.815721,34.160059
...,...,...,...,...,...,...,...
9595,Wyoming,2019-05-01,Transportation Sales per Capita,0.000000,2.991722,0.000000,0.000000
9596,Wyoming,2019-04-01,Transportation Sales per Capita,0.000000,3.034837,0.000000,0.000000
9597,Wyoming,2019-03-01,Transportation Sales per Capita,0.000000,3.317371,0.000000,0.000000
9598,Wyoming,2019-02-01,Transportation Sales per Capita,0.000000,3.158851,0.000000,0.000000


**Nevada Static Bar Chart**

In [10]:
alt.themes.enable('latimes')

nevada = df2_long[df2_long['State'] == 'Nevada']


# Create the percent bar chart
nevada_chart = alt.Chart(nevada).mark_bar(size=12).encode(
    x=alt.X('Date:T', title=''),
    y=alt.Y('Percentage:Q', title='Proportion (%)', scale=alt.Scale(domain=[0, 100])),
    color='Sector:N',
    tooltip=['State', 'Sector', 'Percentage', 'AveragePercentage']
).properties(
    title='Which Sector Consumes the Most Energy in Nevada?',
    width=600,
    height=300
)
# Apply the 538 theme
alt.themes.enable('latimes')


nevada_chart

alt.Chart(...)

**Utah Static Bar Chart**

In [11]:
utah = df2_long[df2_long['State'] == 'Utah']

# Create the percent bar chart
utah_chart = alt.Chart(utah).mark_bar(size=12).encode(
    x=alt.X('Date:T', title=''),
    y=alt.Y('Percentage:Q', title='Proportion (%)', scale=alt.Scale(domain=[0, 100])),
    color='Sector:N',
    tooltip=['State', 'Sector', 'Percentage', 'AveragePercentage']
).properties(
    title='Which Sector Consumes the Most Energy in Utah?',
    width=600,
    height=300
)

utah_chart

alt.Chart(...)

***Interactive Percent Bar Chart***

In [12]:
# Create the percent bar chart
dropdown = alt.binding_select(options=df2_long['State'].unique(), name='Select State: ')
selection = alt.selection_point(fields=['State'], bind=dropdown, value='Nevada')

interact_chart = alt.Chart(df2_long).mark_bar(size=12).encode(
    x=alt.X('Date:T', title=''),
    y=alt.Y('Percentage:Q', title='Proportion (%)', scale=alt.Scale(domain=[0, 100])),
    color='Sector:N',
    tooltip=['State', 'Sector', 'Percentage', 'AveragePercentage']
).properties(
    title='Which Sector Consumes the Most Energy?',
    width=600,
    height=300
).add_selection(
    selection
).transform_filter(
    selection
)

interact_chart

C:\Users\kosti\AppData\Local\Temp\ipykernel_19980\213537593.py:14: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.Chart(...)

***Bar Charts***

In [13]:
#calculate the average percentage of energy consumption by sector for nevada utah and national average

national_avg = df2_long.groupby('Sector')['Percentage'].mean().reset_index()
national_avg

# Create a bar chart for the national average

# remove per capita from the sector column
national_avg['Sector'] = national_avg['Sector'].str.replace(' per Capita', '')

In [14]:
# merge the national average with the nevada average
merged_avg = pd.merge(nevada_avg, national_avg, on='Sector', suffixes=('_Nevada', '_National'))

#rename columns
merged_avg = merged_avg.rename(columns={'Value': 'Nevada', 'Percentage': 'National Average'})

merged_avg

# merge the national average with the utah average
merged_avg = pd.merge(utah_avg, merged_avg, on='Sector')

merged_avg = merged_avg.rename(columns={'Value': 'Utah'})

merged_avg
#create a merged_avg dataset for all states

all_states_avg = df2_long.groupby(['State', 'Sector'])['Percentage'].mean().reset_index()

all_states_avg
# create bar chart for nevada, utah, and national average

merged_avg_long = merged_avg.melt(id_vars='Sector', value_vars=['Nevada', 'Utah', 'National Average'], var_name='State', value_name='Percentage')

merged_avg_long

# Filter out transportation sales data
merged_avg_long = merged_avg_long[merged_avg_long['Sector'] != 'Transportation Sales']


***Static Bar Chart***

In [15]:

# Create a bar chart for the national average
merged_avg_chart = alt.Chart(merged_avg_long).mark_bar(size=60).encode(
    x=alt.X('State:N', title='', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Percentage:Q', title='Proportion (%)', scale=alt.Scale(domain=[0, 50]), axis=alt.Axis(titleFontSize=10)),
    color=alt.Color('State:N', legend=None),
    column=alt.Column('Sector:N', header=alt.Header(labelAngle=0, labelAlign='center', title='Sector')),
).properties(
    title='Utah v. Nevada v. National Sector Breakdown',
    width=300,
    height=400
).configure_axis(
    grid=False
)

merged_avg_chart


alt.Chart(...)

***Interactive Bar Chart***

In [16]:
all_states_avg['Sector'] = all_states_avg['Sector'].str.replace(' per Capita', '')
all_states_avg = all_states_avg[all_states_avg['Sector'] != 'Transportation Sales']

national_avg = national_avg[national_avg['Sector'] != 'Transportation Sales']
# National Chart
national_chart = alt.Chart(national_avg).mark_bar(size=60).encode(
    x=alt.X('Sector:N', title='', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Percentage:Q', title='Proportion (%)', scale=alt.Scale(domain=[0, 50])),
    color='Sector:N'
).properties(
    title='National Average Energy Consumption by Sector',
    width=300,
    height=400
)

# Interactive State Chart
state_dropdown = alt.binding_select(options=list(all_states_avg['State'].unique()), name="Select State: ")
state_selection = alt.selection_single(fields=['State'], bind=state_dropdown, name='State', value='Michigan')

interactive_state_chart = alt.Chart(all_states_avg).mark_bar(size=60).encode(
    x=alt.X('Sector:N', title='', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Percentage:Q', title='Proportion (%)', scale=alt.Scale(domain=[0, 50])),
    color=alt.Color('Sector:N', legend=None),
    tooltip=['State', 'Sector', 'Percentage']
).add_selection(
    state_selection
).transform_filter(
    state_selection
).properties(
    title='State-Specific Energy Consumption by Sector',
    width=300,
    height=400
)

# Combine charts into a single view
combined_chart = alt.hconcat(
    national_chart,
    interactive_state_chart
).configure_axis(
    grid=False
).resolve_scale(
    color='shared'
)

combined_chart


C:\Users\kosti\AppData\Local\Temp\ipykernel_19980\2694886976.py:18: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  state_selection = alt.selection_single(fields=['State'], bind=state_dropdown, name='State', value='Michigan')
C:\Users\kosti\AppData\Local\Temp\ipykernel_19980\2694886976.py:25: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.HConcatChart(...)